In [ ]:
def test_roc(state):
  x=state.copy()
  train=x[(x.index.year>=2000)&(x.index.year<2018)]
  test=x[x.index.year>=2018]

  x1=train['roc_overbought1'] 
  x2=train['roc_overbought2'] 
  x3=train['roc_overbought3'] 
  x4=train['roc_overbought4'] 
  x5=train['roc_overbought5'] 
  x6=train['roc_oversold1'] 
  x7=train['roc_oversold2'] 
  x8=train['roc_oversold3'] 
  x9=train['roc_oversold4'] 
  x10=train['roc_oversold5']
  xx1=test['roc_overbought1'] 
  xx2=test['roc_overbought2'] 
  xx3=test['roc_overbought3'] 
  xx4=test['roc_overbought4'] 
  xx5=test['roc_overbought5'] 
  xx6=test['roc_oversold1'] 
  xx7=test['roc_oversold2'] 
  xx8=test['roc_oversold3'] 
  xx9=test['roc_oversold4'] 
  xx10=test['roc_oversold5'] 
  
  x11=train['rsi_overbought1'] 
  x12=train['rsi_overbought2'] 
  x13=train['rsi_overbought3'] 
  x14=train['rsi_overbought4'] 
  x15=train['rsi_overbought5'] 
  x16=train['rsi_oversold1'] 
  x17=train['rsi_oversold2'] 
  x18=train['rsi_oversold3'] 
  x19=train['rsi_oversold4'] 
  x20=train['rsi_oversold5'] 
  xx11=test['rsi_overbought1'] 
  xx12=test['rsi_overbought2'] 
  xx13=test['rsi_overbought3'] 
  xx14=test['rsi_overbought4'] 
  xx15=test['rsi_overbought5'] 
  xx16=test['rsi_oversold1'] 
  xx17=test['rsi_oversold2'] 
  xx18=test['rsi_oversold3'] 
  xx19=test['rsi_oversold4'] 
  xx20=test['rsi_oversold5'] 

  x21=train['kdj_overbought1'] 
  x22=train['kdj_overbought2'] 
  x23=train['kdj_overbought3'] 
  x24=train['kdj_oversold1'] 
  x25=train['kdj_oversold2'] 
  x26=train['kdj_oversold3'] 
  xx21=test['kdj_overbought1'] 
  xx22=test['kdj_overbought2'] 
  xx23=test['kdj_overbought3'] 
  xx24=test['kdj_oversold1'] 
  xx25=test['kdj_oversold2'] 
  xx26=test['kdj_oversold3'] 

  x27=train['MACD_signal_buy'] 
  x28=train['MACD_signal_sell'] 
  xx27=test['MACD_signal_buy'] 
  xx28=test['MACD_signal_sell'] 

  y=train['nextday_re']/100
  yy=test['nextday_re']/100
  time=list(test.index)
#****************************************************************************************************#
  def fit_evaluation(individual):
    func = toolbox.compile(expr=individual)
    cum = 1
    for i in range(len(x11)):
        state = func(x1[i],x2[i],x3[i],x4[i],x5[i],x6[i],x7[i],x8[i],x9[i],x10[i],
            x11[i],x12[i],x13[i],x14[i],x15[i],x16[i],x17[i],x18[i],x19[i],x20[i],
            x21[i],x22[i],x23[i],x24[i],x25[i],x26[i],x27[i],x28[i])
        if int(state) == 0:   # buy bond
            re = 0.03/252
        if int(state) == 1:     # long Russell 3000
            re = y[i]
        if int(state) == -1:
            re = -y[i]     # short Russell 3000
        cum = cum * (1+re)
    return (cum,)

  toolbox.register('evaluate',fit_evaluation)
  toolbox.register('select',tools.selTournament,tournsize = 3)
  toolbox.register('mate',gp.cxOnePoint)           
  toolbox.register('expr_mut',gp.genFull,pset=pset,min_ = 3,max_ = 5)
  toolbox.register('mutate',gp.mutUniform,expr=toolbox.expr_mut,pset=pset)   
  toolbox.decorate('mate',gp.staticLimit(key=operator.attrgetter('height'),max_value=17))   
  toolbox.decorate('mutate',gp.staticLimit(key=operator.attrgetter('height'),max_value=17)) 
#********************************************************************************************************#
  cxpb = 0.6      # probability of crossover operation
  mutpb = 0.2     # probability of mutation operation
  ngen = 100       # generations
  popSize = 100   # population size
  random.seed(666)

  pop = toolbox.population(n=popSize)
  hof = tools.HallOfFame(1)

  stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
  stats_size = tools.Statistics(len)
  mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
  mstats.register("avg", np.mean)
  mstats.register("std", np.std)
  mstats.register("min", np.min)
  mstats.register("max", np.max)

  pop, log = algorithms.eaSimple(pop, toolbox, cxpb, mutpb, ngen, stats=mstats, halloffame=hof, verbose=True)
#***************************************SHOW TEST RESULT****************************************#  
  func = toolbox.compile(expr = hof.items[0])
  retu = []
  for i in range(len(xx11)):
    state = func(xx1[i],xx2[i],xx3[i],xx4[i],xx5[i],xx6[i],xx7[i],xx8[i],xx9[i],xx10[i],
           xx11[i],xx12[i],xx13[i],xx14[i],xx15[i],xx16[i],xx17[i],xx18[i],xx19[i],xx20[i],
           xx21[i],xx22[i],xx23[i],xx24[i],xx25[i],xx26[i],xx27[i],xx28[i])
    if int(state) == 0:
        re = 0.03/252
    if int(state) == 1:
        re = yy[i]
    if int(state) == -1:
        re = -yy[i]
    retu.append(re)  
#*************************************SHOW STRATEGY**********************************************#
  strategy=[]
  for i in range(len(xx11)):
    state = func(xx1[i],xx2[i],xx3[i],xx4[i],xx5[i],xx6[i],xx7[i],xx8[i],xx9[i],xx10[i],
           xx11[i],xx12[i],xx13[i],xx14[i],xx15[i],xx16[i],xx17[i],xx18[i],xx19[i],xx20[i],
           xx21[i],xx22[i],xx23[i],xx24[i],xx25[i],xx26[i],xx27[i],xx28[i])
    strategy.append(state)
  
  return time,retu,strategy 